### Course's Scope
- Understand the power of API calls to LLM for building software applications
- Best practices for prompting: 2 principles and 6 tactics
- Use cases: summarize, infer, transform, expand
- Buil a chatbot

### Two types of LLMs
1. Base LLM: predicts next word, based on text training data (e.g., you give a question, and the model returns a list of similar questions).
2. Instruction-tuned LLM: follow instructions (e.g., you give a question, the model gives the answer). 
    - RLHF: Reinforcement learning with human feedback. 
    - Trained to be helpful, honest, and harmless. 
    - Recommended. 

### Two Principles
1. Write clear and specific instructions
2. Give the model time to think


In [1]:
#pip install --upgrade pip
#!pip install openai
#!pip install --upgrade openai

In [4]:
#!pip install python-dotenv
#Save API key in the .env `OPENAI_API_KEY` (OPENAI_API_KEY=`your API key`)

In [4]:
#set up API key (need to have credits in your Open AI account)
# Check https://github.com/openai/openai-python/discussions/742

from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(api_key = os.getenv("OPEN_API_KEY"))


In [2]:
#Get output. Check https://github.com/openai/openai-python for updates

def get_completion(prompt, model = "gpt-3.5-turbo"):
    
    messages = [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
    model = model,
    messages = messages,
    temperature = 0.5, 
    )
    return response.choices[0].message.content

#### Principle 1 - Tactic 1
* Use delimiters to clearly indicate parts of the input. Examples: """, ```, ---, < >, XML tags.
* This tactic can help prevent prompt injection. 

In [18]:
# achieve: summarize this paragraph
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a short, single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

To guide a model towards the desired output and reduce irrelevant or incorrect responses, clear and specific instructions should be provided, which may require longer prompts for more clarity and context.


#### Principle 1 - Tactic 2
* Ask for a structured output, e.g., HTML, JSON

In [7]:
prompt = f"""
Generate a list of five made-up song titles along \ 
with their singer and genres. 
Provide them in JSON format with the following keys: 
song_id, song_name, singer, genre.
"""
response = get_completion(prompt)
print(response)

{
  "songs": [
    {
      "song_id": 1,
      "song_name": "Midnight Whispers",
      "singer": "Luna Silver",
      "genre": "Indie Pop"
    },
    {
      "song_id": 2,
      "song_name": "Fading Memories",
      "singer": "Eclipse Dreams",
      "genre": "Electronic"
    },
    {
      "song_id": 3,
      "song_name": "Lost in the Stars",
      "singer": "Aurora Sky",
      "genre": "Alternative Rock"
    },
    {
      "song_id": 4,
      "song_name": "Dreamscape Symphony",
      "singer": "Nova Harmonies",
      "genre": "Orchestral"
    },
    {
      "song_id": 5,
      "song_name": "Neon Nights",
      "singer": "Stellar Beats",
      "genre": "Synthwave"
    }
  ]
}


#### Principle 1 - Tactic 3
* Check whether conditions are satisfied (if... then...else...).

In [8]:
text_1 = f"""
To start, mix the eggs, yogurt, feta cheese in large bowl until creamy. Add the water, baking soda, and a couple tablespoons of avocado oil. Mix well and and set aside.
Grab the biggest baking pan you have, ideally round. You may have to use two 9×16 baking dishes if you don;t have a large round one. Spoon a bit of mixture on the bottom of the baking dish and spread it thin from edge to edge.
Grab one layer of phyllo dough and gently bundle it together like a ball and fill the bottom of the baking dish with a bunch of these balls. Spoon enough egg mixture over to make it wet and repeat with another layer of phyllo balls covered in egg mixture.
Top with 3 sheets of phyllo, each one covered with egg mixture then add some slices of butter all over the top. Allow the banitsa to rest for 15 minutes and preheat the oven to 420F.
Bake for 25-30 minutes, or until the banitsa is deep golden brown. If it gets too much color while baking, cover with tin foil or parchment paper.
When the banitsa comes out of the oven, flip it over and allow it to rest upside down for 20 minutes before eating. You will need to rest the pan on two bowls or something so the pan can hang and the banitsa doesn’t get smashed. Enjoy!
"""

prompt_1 = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - ...
...
Step N - ...

If the text does not contain a sequence of instructions, 
then simply write \"No steps provided.\"
\"\"\"{text_1}\"\"\"
"""

response = get_completion(prompt_1)
print("Completion for Recipe 1:")
print(response)

Completion for Recipe 1:
Step 1 - Mix the eggs, yogurt, feta cheese in large bowl until creamy. Add the water, baking soda, and a couple tablespoons of avocado oil. Mix well and set aside.
Step 2 - Grab the biggest baking pan you have, ideally round. You may have to use two 9×16 baking dishes if you don't have a large round one. Spoon a bit of mixture on the bottom of the baking dish and spread it thin from edge to edge.
Step 3 - Grab one layer of phyllo dough and gently bundle it together like a ball and fill the bottom of the baking dish with a bunch of these balls. Spoon enough egg mixture over to make it wet and repeat with another layer of phyllo balls covered in egg mixture.
Step 4 - Top with 3 sheets of phyllo, each one covered with egg mixture then add some slices of butter all over the top. Allow the banitsa to rest for 15 minutes and preheat the oven to 420F.
Step 5 - Bake for 25-30 minutes, or until the banitsa is deep golden brown. If it gets too much color while baking, co

#### Principle 1 - Tactic 4
* Few-shot prompting: Give successful example tasks before asking the model to perform a task

In [5]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a mighty oak tree standing tall through storms and adversity. It is the unwavering determination to bounce back from setbacks, to rise above challenges, and to grow stronger with each trial. Just as the oak tree bends but never breaks, resilience allows us to persevere and thrive in the face of adversity.


#### Principle 2 - Tactic 1
* Specify the steps to complete a task. Recommend to ask for output in a specified format.

In [8]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""

# example 2 - structured

prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill, siblings, embark on a quest to fetch water from a hilltop well, but encounter misfortune along the way. 
Translation: Jack et Jill, frère et sœur, se lancent dans une quête pour chercher de l'eau d'un puits au sommet d'une colline, mais rencontrent des malheurs en chemin.
Names: Jack, Jill
Output JSON: {"french_summary": "Jack et Jill, frère et sœur, se lancent dans une quête pour chercher de l'eau d'un puits au sommet d'une colline, mais rencontrent des malheurs en chemin.", "num_names": 2}


#### Principle 2 - Tactic 2
* Teach the model to work out its own solution before rushing to a conclusion.

In [9]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct. They correctly identified the costs for land, solar panels, and maintenance, and calculated the total cost by adding them together.


In [10]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

# The answer is still incorrect, which is inconsistent from what the course result shows. 

To calculate the total cost for the first year of operations, we need to consider the costs of land, solar panels, and maintenance.

1. Land cost: $100 / square foot
The cost of land is calculated by multiplying the size of the installation (in square feet) by the cost per square foot.
Land cost = $100 * x (where x is the size of the installation in square feet)

2. Solar panel cost: $250 / square foot
The cost of solar panels is calculated by multiplying the size of the installation (in square feet) by the cost per square foot.
Solar panel cost = $250 * x (where x is the size of the installation in square feet)

3. Maintenance cost: $100,000 + $10 / square foot
The maintenance cost is a flat fee of $100,000 per year, plus an additional cost per square foot.
Maintenance cost = $100,000 + $10 * x (where x is the size of the installation in square feet)

Total cost for the first year of operations = Land cost + Solar panel cost + Maintenance cost
Total cost = ($100 * x) + ($250 * x) + ($

### Model Limitation
- Hallucinations: fabricated ideas

In [14]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie, by three sentences:
1...
2...
3...
"""
response = get_completion(prompt)
print(response)

1. The AeroGlide UltraSlim Smart Toothbrush by Boie is a technologically advanced toothbrush designed to provide a superior cleaning experience. It features ultra-soft bristles that are gentle on gums while effectively removing plaque and debris from teeth.

2. This smart toothbrush is equipped with a built-in timer and pressure sensor to ensure proper brushing technique and prevent over-brushing. It also connects to a mobile app via Bluetooth, allowing users to track their brushing habits and receive personalized recommendations for improving oral health.

3. With its sleek and slim design, the AeroGlide UltraSlim Smart Toothbrush is not only effective but also aesthetically pleasing. It is made from durable and eco-friendly materials, making it a sustainable choice for oral care.
